In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
# fix random seed for reproducibility
tf.random.set_seed(7)

#Loading Dataset

In [7]:
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

#Truncate and pad input sequences

In [8]:
max_review_length = 1000
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

#Building Model

In [9]:
def ModelBuilder():
  embedding_vecor_length = 32
  model = Sequential()
  model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
  model.add(Dropout(0.2))
  model.add(LSTM(100))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
  model.fit(X_train, y_train, epochs=10, batch_size=64,callbacks=[early_stopping], validation_split=0.1)
  # Final evaluation of the model
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("Accuracy: %.2f%%" % (scores[1]*100))
  return model

In [10]:
model=ModelBuilder()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.6233 - loss: 0.6217 - val_accuracy: 0.8464 - val_loss: 0.3653
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - accuracy: 0.8458 - loss: 0.3504 - val_accuracy: 0.8708 - val_loss: 0.3248
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - accuracy: 0.8981 - loss: 0.2623 - val_accuracy: 0.8788 - val_loss: 0.3500
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9096 - loss: 0.2298 - val_accuracy: 0.8736 - val_loss: 0.3664
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 14s 39ms/step - accuracy: 0.9324 - loss: 0.1870 - val_accuracy: 0.8576 - val_loss: 0.3818
Accuracy: 86.66%


#Testing Model

In [16]:
test_reviews = [
        "This movie was absolutely fantastic! I loved every minute of it.",
        "Terrible film. Waste of time and money. Very disappointing.",
        "The movie was very good",
        "Outstanding performance by the lead actor. Highly recommended!",
        "I fell asleep halfway through. Boring and poorly written."
    ]

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def encode_review(text, word_index, top_words, max_length):
    words = text.lower().split()
    encoded = []
    for word in words:
        word_id = word_index.get(word, 0) + 3
        if word_id < top_words:
            encoded.append(word_id)
        else:
            encoded.append(0)
    return sequence.pad_sequences([encoded], maxlen=max_length)

print("\n--- Predicting on custom reviews ---")
for i, review_text in enumerate(test_reviews):
    encoded_review = encode_review(review_text, word_index, top_words, max_review_length)
    prediction = model.predict(encoded_review)
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
    print(f"Review {i+1}: '{review_text}'")
    print(f"Prediction: {prediction[0][0]:.4f}, Sentiment: {sentiment}\n")



--- Predicting on custom reviews ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Review 1: 'This movie was absolutely fantastic! I loved every minute of it.'
Prediction: 0.5061, Sentiment: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Review 2: 'Terrible film. Waste of time and money. Very disappointing.'
Prediction: 0.0747, Sentiment: Negative

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Review 3: 'The movie was very good'
Prediction: 0.5542, Sentiment: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Review 4: 'Outstanding performance by the lead actor. Highly recommended!'
Prediction: 0.8910, Sentiment: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Review 5: 'I fell asleep halfway through. Boring and poorly written.'
Prediction: 0.0201, Sentiment: Negative

